# This notebook contains scripts used to plot single station pair examples.

In [1]:
import os,random,time,pickle
import numpy as np
import matplotlib.pyplot as plt
from obspy import UTCDateTime
from obspy.signal.filter import bandpass,highpass,lowpass
from seisgo import noise,utils,dispersion,monitoring

## Global parameters

In [2]:
figformat="tiff"
figdpi=200
dlabels=["Raw","One-bit"]
figlabels=["(a)","(b)","(c)","(d)","(e)","(f)","(g)","(h)"]
cmap="seismic"
methods=["linear","robust","selective","cluster","pws","tfpws","nroot","acf"]
method_labels=["Linear","Robust","Selective","Cluster","PWS","tf-PWS","$N^{th}$-root","ACF"]
colors=["k","b","c","y","g","orange","r","m"]

dataroot='data_stacking'

######for XZ examples
source_xz="XZ.A02"
receiver_xz="XZ.A24"
network_xz="XZ"
comp="ZZ"
pair_xz=source_xz+"_"+receiver_xz
raw_xz=os.path.join(dataroot,"MERGED_"+network_xz+"_raw")
datadir_raw_xz=os.path.join(raw_xz,source_xz)
dfile_raw_xz=datadir_raw_xz+"/"+pair_xz+".h5"
cdata_raw_xz=noise.extract_corrdata(dfile_raw_xz,pair=pair_xz,comp=comp)[pair_xz][comp]
tnorm_xz=os.path.join(dataroot,"MERGED_"+network_xz+"_tnorm")
datadir_tnorm_xz=os.path.join(tnorm_xz,source_xz)
dfile_tnorm_xz=datadir_tnorm_xz+"/"+pair_xz+".h5"
cdata_tnorm_xz=noise.extract_corrdata(dfile_tnorm_xz,pair=pair_xz,comp=comp)[pair_xz][comp]

############ for cascadia examples
source="7D.J33A"
receiver="TA.G03D"
network="cascadia"
pair=source+"_"+receiver
raw=os.path.join(dataroot,"MERGED_"+network+"_raw")
datadir_raw=os.path.join(raw,source)
dfile_raw=datadir_raw+"/"+pair+".h5"
cdata_raw=noise.extract_corrdata(dfile_raw,pair=pair,comp=comp)[pair][comp]
tnorm=os.path.join(dataroot,"MERGED_"+network+"_tnorm")
datadir_tnorm=os.path.join(tnorm,source)
dfile_tnorm=datadir_tnorm+"/"+pair+".h5"
cdata_tnorm=noise.extract_corrdata(dfile_tnorm,pair=pair,comp=comp)[pair][comp]

## Stacking examples for the above station pairs

In [3]:
savefig=True
figsize=(12,9)
snr_db=True

plt.figure(figsize=figsize,facecolor='w')
"""
1. Cascadia examples.
"""
lag=300
vmin=2 
vmax=4.5
snr_window_extend=100
noise_window_offset=30
freqmin,freqmax=[0.1,.4]#[0.04,0.4]

scale=0.45

with open('xcorr_stacks_singlepair_stack_all_'+network+'.pk','rb') as xf:
    stack_all=pickle.load(xf)
    
stack_snr_all={dlabels[0]:np.ndarray((2,len(method_labels))),dlabels[1]:np.ndarray((2,len(method_labels)))}
for i,c in enumerate([cdata_raw,cdata_tnorm]):
    netstachan1 = c.net[0]+'.'+c.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
    netstachan2 = c.net[1]+'.'+c.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]

    dt,maxlag,dist,ttime,substack = [c.dt,c.lag,c.dist,\
                                            c.time,c.substack]
   # lags for display
    if lag>maxlag:raise ValueError('lag excceds maxlag!')
    tvec=np.arange(-maxlag,maxlag+0.5*dt,dt)
    # t is the time labels for plotting
    tstep=int(int(lag)/5)
    t1=np.arange(-int(lag),0,step=tstep);t2=np.arange(0,int(lag+0.5*tstep),step=tstep)
    t=np.concatenate((t1,t2))

    """
    Begin plot stacking
    """
    plt.subplot(2,2,i+1)
    for j,m in enumerate(methods):
#         ds = stack_all_xz[dlabels[i]][j]
        ds = bandpass(stack_all[dlabels[i]][j],freqmin,freqmax,1/dt,corners=4, zerophase=True)
        # measure the SNR
        snr,widx_p,widx_n=utils.get_snr(ds,tvec,dist,vmin,vmax,offset=noise_window_offset,
                                        getwindow=True,extend=snr_window_extend,db=snr_db)
        stack_snr_all[dlabels[i]][:,j]=snr
        plt.plot(tvec,j+scale*ds/np.max(np.abs(ds[widx_p[0][0]:widx_p[0][1]])),color=colors[j])
        
        if j==0:
            plt.text(-0.95*lag,j-0.25,"SNR-neg=%4.1f"%(snr[0]))
            plt.text(0.48*lag,j-0.25,"SNR-pos=%4.1f"%(snr[1]))
        else:
            plt.text(-0.9*lag,j-0.25,"%4.1f"%(snr[0]))
            plt.text(0.75*lag,j-0.25,"%4.1f"%(snr[1]))

    plt.ylim([len(methods)-0.3,-1])
    #plot SNR measuring windows.
    snrbar_y=-0.65
    plt.plot(tvec[widx_p[0]],[snrbar_y,snrbar_y],'k-',linewidth=2)
    plt.plot(tvec[widx_p[1]],[snrbar_y,snrbar_y],'k--',linewidth=2)
    plt.plot(tvec[widx_n[0]],[snrbar_y,snrbar_y],'k-',linewidth=2)
    plt.plot(tvec[widx_n[1]],[snrbar_y,snrbar_y],'k--',linewidth=2)
    
    plt.text(np.mean(tvec[widx_p[0]]),snrbar_y-0.1,'signal',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_p[1]]),snrbar_y-0.1,'noise',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_n[0]]),snrbar_y-0.1,'signal',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_n[1]]),snrbar_y-0.1,'noise',horizontalalignment='center')
    
    plt.xlim([-lag,lag])
    plt.yticks(range(len(methods)),labels=method_labels,fontsize=12)
    plt.xlabel('lag time (s)',fontsize=12)
    plt.title('%s %s-%s: %d km: %s' % (figlabels[i],netstachan1,netstachan2,
                                                               dist,dlabels[i]),
                fontsize=14,loc="left",pad=9)
    plt.text(0.45*lag,len(methods)-0.5,"%4.1f-%4.1f Hz"%(freqmin,freqmax),
            fontsize=12)

    plt.plot((0,0),(len(methods)-0.3,-1),'k-')

"""
2. XZ examples.
"""
lag_xz=150
vmin_xz=2 
vmax_xz=6
snr_window_extend_xz=20
noise_window_offset_xz=30
freqmin_xz,freqmax_xz=[0.1,.4]#[0.04,0.4]

scale_xz=0.45
with open('xcorr_stacks_singlepair_stack_all_'+network_xz+'.pk','rb') as xf:
    stack_all_xz=pickle.load(xf)
    
stack_snr_all_xz={dlabels[0]:np.ndarray((2,len(method_labels))),dlabels[1]:np.ndarray((2,len(method_labels)))}
for i,c in enumerate([cdata_raw_xz,cdata_tnorm_xz]):
    netstachan1 = c.net[0]+'.'+c.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
    netstachan2 = c.net[1]+'.'+c.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]

    dt,maxlag,dist,ttime,substack = [c.dt,c.lag,c.dist,\
                                            c.time,c.substack]
   # lags for display
    if lag_xz>maxlag:raise ValueError('lag excceds maxlag!')
    tvec=np.arange(-maxlag,maxlag+0.5*dt,dt)
    # t is the time labels for plotting
    tstep=int(int(lag_xz)/5)
    t1=np.arange(-int(lag_xz),0,step=tstep);t2=np.arange(0,int(lag_xz+0.5*tstep),step=tstep)
    t=np.concatenate((t1,t2))

    """
    Begin plot stacking
    """
    plt.subplot(2,2,i+3)
    for j,m in enumerate(methods):
#         ds = stack_all_xz[dlabels[i]][j]
        ds = bandpass(stack_all_xz[dlabels[i]][j],freqmin_xz,freqmax_xz,1/dt,corners=4, zerophase=True)
        # measure the SNR
        snr,widx_p,widx_n=utils.get_snr(ds,tvec,dist,vmin_xz,vmax_xz,offset=noise_window_offset_xz,
                                        getwindow=True,extend=snr_window_extend_xz,db=snr_db)
        stack_snr_all_xz[dlabels[i]][:,j]=snr
        plt.plot(tvec,j+scale_xz*ds/np.max(np.abs(ds[widx_p[0][0]:widx_p[0][1]])),color=colors[j])
        
        if j==0:
            plt.text(-0.95*lag_xz,j-0.25,"SNR-neg=%4.1f"%(snr[0]))
            plt.text(0.48*lag_xz,j-0.25,"SNR-pos=%4.1f"%(snr[1]))
        else:
            plt.text(-0.9*lag_xz,j-0.25,"%4.1f"%(snr[0]))
            plt.text(0.75*lag_xz,j-0.25,"%4.1f"%(snr[1]))

    plt.ylim([len(methods)-0.3,-1])
    #plot SNR measuring windows.
    snrbar_y=-0.65
    plt.plot(tvec[widx_p[0]],[snrbar_y,snrbar_y],'k-',linewidth=2)
    plt.plot(tvec[widx_p[1]],[snrbar_y,snrbar_y],'k--',linewidth=2)
    plt.plot(tvec[widx_n[0]],[snrbar_y,snrbar_y],'k-',linewidth=2)
    plt.plot(tvec[widx_n[1]],[snrbar_y,snrbar_y],'k--',linewidth=2)
    
    plt.text(np.mean(tvec[widx_p[0]]),snrbar_y-0.1,'signal',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_p[1]]),snrbar_y-0.1,'noise',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_n[0]]),snrbar_y-0.1,'signal',horizontalalignment='center')
    plt.text(np.mean(tvec[widx_n[1]]),snrbar_y-0.1,'noise',horizontalalignment='center')
    
    plt.xlim([-lag_xz,lag_xz])
    plt.yticks(range(len(methods)),labels=method_labels,fontsize=12)
    plt.xlabel('lag time (s)',fontsize=12)
    plt.title('%s %s-%s: %d km: %s' % (figlabels[i+2],netstachan1,netstachan2,
                                                               dist,dlabels[i]),
                fontsize=14,loc="left",pad=9)
    plt.text(0.45*lag_xz,len(methods)-0.5,"%4.1f-%4.1f Hz"%(freqmin_xz,freqmax_xz),
            fontsize=12)

    plt.plot((0,0),(len(methods)-0.3,-1),'k-')
    
plt.tight_layout()
if savefig:
    plt.savefig("xcorr_stacks_singlepair_twonetworks."+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()

## Convergence rate of all stacking methods
Increase from 5 to the whole length, randomly resample a subset of the whole NCFs. All method will use the same randomly sampled data set for stacking. However, each stacking result will compare with the stack of whole dataset using the same method.

In [5]:
step=5
nstart=step

savefig=True
figsize=(11,6)
plt.figure(figsize=figsize,facecolor='w')

"""
1. Cascadia examples
"""
c_temp=cdata_raw.subset(endtime="2012_04_1_0_0_0",overwrite=False)
ntrace=c_temp.data.shape[0]
stepall=np.arange(nstart,ntrace+0.5*step,step)
with open('xcorr_stacks_singlepair_convergence_step%d_%s.pk'%(step,network),'rb') as xf:
    stack_convergence=pickle.load(xf)
# timeflag
tdays=[]
t0=c_temp.time[0]
for k,s in enumerate(stepall):
    tdays.append((c_temp.time[int(s)-1]-t0)/24/3600)
for i in range(len(dlabels)):
    plt.subplot(2,2,i+1)
    for j,m in enumerate(methods):
        plt.plot(tdays,stack_convergence[dlabels[i]][j,:],color=colors[j],label=method_labels[j])
    plt.grid("both")
#     plt.ylim([0,1])
    plt.yticks(fontsize=12)
    plt.ylim([0,1])
    plt.xlim([tdays[0],tdays[-1]])
#     plt.xlim([100*nstart/ntrace,100])
#     plt.xlim([stepall[0],stepall[-1]])
    plt.xlabel("time (days)",fontsize=13)
    if i==0:plt.ylabel("CC with total stack",fontsize=12)
    if i==0:plt.legend(loc="lower right",ncol=2,fontsize=12,columnspacing=0.5,
                      labelspacing=0.3)
    plt.title(figlabels[i]+" "+cdata_raw.id.replace("..BHZ","").replace("_","-")+": "+dlabels[i],fontsize=14,loc='left',pad=9)

    
"""
2. XZ examples
"""
ntrace=cdata_raw_xz.data.shape[0]
stepall=np.arange(nstart,ntrace+0.5*step,step)
with open('xcorr_stacks_singlepair_convergence_step%d_%s.pk'%(step,network_xz),'rb') as xf:
    stack_convergence_xz=pickle.load(xf)
tdays_xz=[]
t0=cdata_raw_xz.time[0]
for k,s in enumerate(stepall):
    tdays_xz.append((cdata_raw_xz.time[int(s)-1]-t0)/24/3600)
    
for i in range(len(dlabels)):
    plt.subplot(2,2,i+3)
    for j,m in enumerate(methods):
        plt.plot(tdays_xz,stack_convergence_xz[dlabels[i]][j,:],color=colors[j],label=method_labels[j])
    plt.grid("both")
#     plt.ylim([0,1])
    plt.yticks(fontsize=12)
    plt.ylim([0,1])
    plt.xlim([tdays_xz[0],tdays_xz[-1]])
#     plt.xlim([100*nstart/ntrace,100])
#     plt.xlim([stepall[0],stepall[-1]])
    plt.xlabel("time (days)",fontsize=13)
    if i==0:plt.ylabel("CC with total stack",fontsize=12)
#     if i==1:plt.legend(bbox_to_anchor=(1.01, 1.0),ncol=1,fontsize=11)
    plt.title(figlabels[i+2]+" "+cdata_raw_xz.id.replace("..BHZ","").replace("_","-")+": "+dlabels[i],fontsize=14,loc='left',pad=9)
    
plt.tight_layout()
if savefig:
    plt.savefig("xcorr_stacks_singlepair_convergence_twonetworks."+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()
        

## Seismic monitoring: dv/v results
### Assemble dv/v results first

In [6]:
freq = [0.1,0.4]                              #targeted frequency band for monitoring
"""
1. Cascadia examples.
"""
win_len = 3600*96                           #Window length i.e. Resolution
offset=1.7/freq[0]
vmin = 1.5                                    #minimum velocity of the direct waves -> 
#start of the coda waveforms
lwin = 2/freq[0]                                    #window length in seconds for the coda waves

dv_max = 2/100                              #limit for the dv/v
vdata_all={dlabels[0]:[],dlabels[1]:[],"time":[]}
vdata_maxcc={dlabels[0]:[],dlabels[1]:[],"time":[]}
nproc=20
endtime="2012_06_1_0_0_0"

for i,c in enumerate([cdata_raw,cdata_tnorm]):
    c_temp=c.subset(endtime=endtime,overwrite=False)
    for j,m in enumerate(methods):
        print("method: %s"%(m))
        vdatatest=monitoring.get_dvv(c_temp,freq,lwin,resolution=win_len,vmin=vmin,dvmax=dv_max,
                                     offset=offset,stack_method=m,nproc=nproc,plot=False,method="ts")
        vdata_all[dlabels[i]].append(vdatatest.data2)
        vdata_maxcc[dlabels[i]].append(vdatatest.maxcc2)
    vdata_all["time"].append(vdatatest.time)
    vdata_maxcc["time"].append(vdatatest.time)

"""
2. XZ examples.
"""
win_len = 3600*96                           #Window length i.e. Resolution
offset_xz=-1.5/freq[0]  #negative offset will make it use the ballistic phase.
vmin_xz = 2              #minimum velocity of the direct waves -> 
#start of the coda waveforms
lwin_xz = lwin                   #window length in seconds for the coda waves

vdata_all_xz={dlabels[0]:[],dlabels[1]:[],"time":[]}
vdata_maxcc_xz={dlabels[0]:[],dlabels[1]:[],"time":[]}

for i,c in enumerate([cdata_raw_xz,cdata_tnorm_xz]):
    for j,m in enumerate(methods):
        print("method: %s"%(m))
        vdatatest=monitoring.get_dvv(c,freq,lwin_xz,resolution=win_len,vmin=vmin_xz,dvmax=dv_max,
                                     offset=offset_xz,stack_method=m,nproc=nproc,plot=False,method="ts")
        vdata_all_xz[dlabels[i]].append(vdatatest.data2)
        vdata_maxcc_xz[dlabels[i]].append(vdatatest.maxcc2)
    vdata_all_xz["time"].append(vdatatest.time)
    vdata_maxcc_xz["time"].append(vdatatest.time)

method: linear
working on 57 windows with 20 workers.
method: robust
working on 57 windows with 20 workers.
method: selective
working on 57 windows with 20 workers.
method: cluster
working on 57 windows with 20 workers.
method: pws
working on 57 windows with 20 workers.
method: tfpws
working on 57 windows with 20 workers.
method: nroot
working on 57 windows with 20 workers.
method: acf
working on 57 windows with 20 workers.
method: linear
working on 57 windows with 20 workers.
method: robust
working on 57 windows with 20 workers.
method: selective
working on 57 windows with 20 workers.
method: cluster
working on 57 windows with 20 workers.
method: pws
working on 57 windows with 20 workers.
method: tfpws
working on 57 windows with 20 workers.
method: nroot
working on 57 windows with 20 workers.
method: acf
working on 57 windows with 20 workers.
method: linear
working on 35 windows with 20 workers.
method: robust
working on 35 windows with 20 workers.
method: selective
working on 35 wind

### Plot dv/v results 

In [7]:
savefig=True
plt.figure(figsize=(11.5,7),facecolor="w")
ccmin=0.6  #minimum correlation coefficient after stretching. maxcc1 and maxcc2 attributes in DvvData.
"""
1. Cascadia examples.
"""
for i in range(len(dlabels)):
    plt.subplot(2,2,i+1)
    plt.hlines(0,0,len(vdata_all[dlabels[i]][0]),colors="k")
    plt.xlim([0,len(vdata_all[dlabels[i]][0])])
    plt.ylim([-1.5,1.5])
    nodatalist=[]
    for j,m in enumerate(methods):
        vdtemp=vdata_all[dlabels[i]][j]
        vdtemp[vdata_maxcc[dlabels[i]][j]<ccmin]=np.nan
        if np.isnan(vdtemp).all():
            print('cascadia: all nan for '+m)
            nodatalist.append(method_labels[j])
        plt.plot(vdtemp,color=colors[j],label=method_labels[j],alpha=.8)
    #
    xticks=np.arange(0,len(vdata_all[dlabels[i]][0]),10)
    xticklabels=[]
    for ii in range(len(xticks)):
        xticklabels.append(str(UTCDateTime(vdata_all['time'][i][xticks[ii]]))[:10])
    plt.grid()
    plt.xticks(xticks,labels=xticklabels,fontsize=12,rotation=20)
    plt.yticks(fontsize=13)
    if i==0:plt.ylabel('dv/v (%)',fontsize=13)
    if len(nodatalist)>0:
        plt.text(xticks[-1]*0.6,-1,"no data for "+str(nodatalist),fontsize=13,backgroundcolor="w")
    if i==1:plt.legend(bbox_to_anchor=(1.02, 1),fontsize=11)
    plt.text(xticks[-1]*0.05,1.2,"%4.1f-%4.1f Hz"%(freq[0],freq[1]),fontsize=13,backgroundcolor="w")
    plt.title(figlabels[i]+" "+cdata_raw.id.replace("..BHZ","").replace("_","-")+\
              ": "+dlabels[i]+": CC >="+str(ccmin),fontsize=14,loc='left',pad=9)

"""
2. XZ examples.
"""
for i in range(len(dlabels)):
    plt.subplot(2,2,i+3)
    plt.hlines(0,0,len(vdata_all[dlabels[i]][0]),colors="k")
    plt.xlim([0,len(vdata_all_xz[dlabels[i]][0])])
    plt.ylim([-1.,1.3])
    nodatalist=[]
    for j,m in enumerate(methods):
        vdtemp=vdata_all_xz[dlabels[i]][j]
        vdtemp[vdata_maxcc_xz[dlabels[i]][j]<ccmin]=np.nan
        if np.isnan(vdtemp).all():
            print('XZ: all nan for '+m)
            nodatalist.append(method_labels[j])
        plt.plot(vdtemp,color=colors[j],label=method_labels[j],alpha=.8)
    #
    xticks=np.arange(0,len(vdata_all_xz[dlabels[i]][0]),6)
    xticklabels=[]
    for ii in range(len(xticks)):
        xticklabels.append(str(UTCDateTime(vdata_all_xz['time'][i][xticks[ii]]))[:10])
    plt.grid()
    plt.xticks(xticks,labels=xticklabels,fontsize=12,rotation=20)
    plt.yticks(fontsize=13)
    if i==0:
        plt.ylabel('dv/v (%)',fontsize=13)
    if len(nodatalist)>0:
        plt.text(xticks[-1]*0.6,-.6,"no data: "+str(nodatalist),fontsize=13,backgroundcolor="w")
    if i==1:plt.legend(bbox_to_anchor=(1.02, 1),fontsize=11)
    plt.text(xticks[-1]*0.05,1.,"%4.1f-%4.1f Hz"%(freq[0],freq[1]),fontsize=13,backgroundcolor="w")
    
    plt.title(figlabels[i+2]+" "+cdata_raw_xz.id.replace("..BHZ","").replace("_","-")+\
              ": "+dlabels[i]+": CC >="+str(ccmin),
              fontsize=14,loc='left',pad=9)
    
plt.tight_layout()
if savefig:
    plt.savefig("dvv_examples_twonetworks."+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()


XZ: all nan for pws


## Xcorr examples of the two station pairs with dv/v windows

In [8]:
savefig=True
"""
1. Cascadia example.
"""
#####
tmin = offset+cdata_raw.dist/vmin
twin = [tmin,tmin+lwin]

c_temp=cdata_raw.subset(endtime=endtime,overwrite=False)
nwin_dvv=c_temp.data.shape[0]

lag=180
freqmin,freqmax=[0.1,0.4]
figsize=(12,8)
plt.figure(figsize=figsize,facecolor='w')
snr_all=[]
for i,c in enumerate([cdata_raw,cdata_tnorm]):
    netstachan1 = c.net[0]+'.'+c.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
    netstachan2 = c.net[1]+'.'+c.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]

    dt,maxlag,dist,ttime,substack = [c.dt,c.lag,c.dist,\
                                            c.time,c.substack]
    
   # lags for display
    if not lag:lag=maxlag
    if lag>maxlag:raise ValueError('lag excceds maxlag!')
    lag0=np.min([1.0*lag,maxlag])
    tvec=np.arange(-lag0,lag0+0.5*dt,dt)
    
    # t is the time labels for plotting
    if lag>=5:
        tstep=int(int(lag)/5)
        t1=np.arange(-int(lag),0,step=tstep);t2=np.arange(0,int(lag+0.5*tstep),step=tstep)
        t=np.concatenate((t1,t2))
    else:
        tstep=lag/5
        t1=np.arange(-lag,0,step=tstep);t2=np.arange(0,lag+0.5*tstep,step=tstep)
        t=np.concatenate((t1,t2))

    indx1 = int((maxlag-lag0)/dt);indx2 = indx1+2*int(lag0/dt)+1

    # cc matrix
    data = np.ndarray.copy(c.data[:,indx1:indx2])
    timestamp = np.empty(ttime.size,dtype='datetime64[s]')
    # print(data.shape)
    nwin = data.shape[0]
    amax = np.zeros(nwin,dtype=np.float32)
    if nwin==0:
        print('continue! no enough trace to plot!')

    tmarks = []
    data_normalizd=np.zeros(data.shape)

    # load cc for each station-pair
    for ii in range(nwin):
        if freqmin is not None and freqmax is not None:
            data[ii] = bandpass(data[ii],freqmin,freqmax,1/dt,corners=4, zerophase=True)
        data[ii] = utils.taper(data[ii]-np.mean(data[ii]),maxlen=10)
        amax[ii] = np.max(np.abs(data[ii]))
        data_normalizd[ii] = data[ii]/amax[ii]
        timestamp[ii] = UTCDateTime(ttime[ii])
        tmarks.append(UTCDateTime(ttime[ii]).strftime('%Y-%m-%dT%H'))

    # plotting
    if nwin>10:
        tick_inc = int(nwin/6)
    else:
        tick_inc = 2

#     plt.subplot(1,6*len(cdata_all),(i*6+1,i*6+4))
    plt.subplot(2,len(dlabels),i+1)
    extent=[-lag0,lag0,nwin,0]

    plt.imshow(data_normalizd,cmap=cmap,extent=extent,aspect='auto')
    
    tstack=np.arange(-lag0,lag0+0.5*dt,dt)
    dstack = np.mean(data,axis=0)
    dstack /= np.max(np.abs(dstack))
    if len(tstack)>len(dstack):tstack=tstack[:-2]
    plt.plot(tstack,-35+30*dstack,'b-',linewidth=1,label="linear stack")
    plt.text(-0.9*lag,-50,"average",color='b')
    # zero line
    plt.plot((0,0),(nwin,-70),'k-')
    plt.ylim([nwin,-70])
    #dvv window boxes
    pwin_indx = np.where((tstack>=np.min(twin))&(tstack<=np.max(twin)))[0]
    plt.plot([tstack[pwin_indx[0]],tstack[pwin_indx[-1]+1],tstack[pwin_indx[-1]+1],
              tstack[pwin_indx[0]],tstack[pwin_indx[0]]],
            [nwin_dvv,nwin_dvv,0,0,nwin_dvv],'c--',lw=3)
    if i==0:
        print('Cascadia window')
        print(tstack[pwin_indx[0]],tstack[pwin_indx[-1]+1])
    plt.text(tstack[pwin_indx[-1]]*1.06,0.5*nwin_dvv,"dv/v\nwindow",
                fontsize=14,ha="left",va="center",color="k")
    if freqmin is not None and freqmax is not None:
        plt.title('%s %s-%s: %d km: %s' % (figlabels[i],netstachan1,netstachan2,
                                                                   dist,dlabels[i]),
                    fontsize=14,loc="left",pad=9)
        plt.text(-0.95*lag,nwin*0.95,"%4.1f-%4.1f Hz"%(freqmin,freqmax),backgroundcolor="w",
                fontsize=12)
    else:
        plt.title('%s %s-%s: %d km: %s' % (figlabels[i],netstachan1,netstachan2,dist,dlabels[i]),
                 fontsize=14,loc="left",pad=9)
        plt.text(-0.95*lag,nwin*0.95,"unfiltered",backgroundcolor="w",fontsize=14)
    plt.xlabel('lag time (s)',fontsize=13)
    plt.xticks(t)
    plt.yticks(np.arange(0,nwin,step=tick_inc),labels=tmarks[0:nwin:tick_inc])
    plt.xlim([-lag,lag])

"""
2. XZ examples
"""
tmin_xz = offset_xz+cdata_raw_xz.dist/vmin_xz
twin_xz = [tmin_xz,tmin_xz+lwin_xz]

# savefig=False
lag=100
freqmin,freqmax=[0.1,.4]
snr_all=[]
for i,c in enumerate([cdata_raw_xz,cdata_tnorm_xz]):
    netstachan1 = c.net[0]+'.'+c.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
    netstachan2 = c.net[1]+'.'+c.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]

    dt,maxlag,dist,ttime,substack = [c.dt,c.lag,c.dist,\
                                            c.time,c.substack]
    
   # lags for display
    if not lag:lag=maxlag
    if lag>maxlag:raise ValueError('lag excceds maxlag!')
    lag0=np.min([1.0*lag,maxlag])
    tvec=np.arange(-lag0,lag0+0.5*dt,dt)
    
    # t is the time labels for plotting
    if lag>=5:
        tstep=int(int(lag)/5)
        t1=np.arange(-int(lag),0,step=tstep);t2=np.arange(0,int(lag+0.5*tstep),step=tstep)
        t=np.concatenate((t1,t2))
    else:
        tstep=lag/5
        t1=np.arange(-lag,0,step=tstep);t2=np.arange(0,lag+0.5*tstep,step=tstep)
        t=np.concatenate((t1,t2))

    indx1 = int((maxlag-lag0)/dt);indx2 = indx1+2*int(lag0/dt)+1

    # cc matrix
    data = np.ndarray.copy(c.data[:,indx1:indx2])
    timestamp = np.empty(ttime.size,dtype='datetime64[s]')
    # print(data.shape)
    nwin = data.shape[0]
    amax = np.zeros(nwin,dtype=np.float32)
    if nwin==0:
        print('continue! no enough trace to plot!')

    tmarks = []
    data_normalizd=np.zeros(data.shape)

    # load cc for each station-pair
    for ii in range(nwin):
        if freqmin is not None and freqmax is not None:
            data[ii] = bandpass(data[ii],freqmin,freqmax,1/dt,corners=4, zerophase=True)
        data[ii] = utils.taper(data[ii]-np.mean(data[ii]),maxlen=10)
        amax[ii] = np.max(np.abs(data[ii]))
        data_normalizd[ii] = data[ii]/amax[ii]
        timestamp[ii] = UTCDateTime(ttime[ii])
        tmarks.append(UTCDateTime(ttime[ii]).strftime('%Y-%m-%dT%H'))

    # plotting
    if nwin>10:
        tick_inc = int(nwin/6)
    else:
        tick_inc = 2

#     plt.subplot(1,6*len(cdata_all),(i*6+1,i*6+4))
    plt.subplot(2,len(dlabels),i+3)
    extent=[-lag0,lag0,nwin,0]

    plt.imshow(data_normalizd,cmap=cmap,extent=extent,aspect='auto')
    
    tstack=np.arange(-lag0,lag0+0.5*dt,dt)
    dstack = np.mean(data,axis=0)
    dstack /= np.max(np.abs(dstack[int(len(dstack)/2 +10)]))
    dstack=utils.taper(dstack)
    if len(tstack)>len(dstack):tstack=tstack[:-2]
    plt.plot(tstack,-15+(3+7*(1-i))*dstack,'b-',linewidth=1,label="linear stack")
    plt.text(-0.9*lag,-20,"average",color='b',fontsize=12)
    # zero line
    plt.vlines(0,-30,nwin,colors='k')
    plt.ylim([nwin,-30])
    #dvv window boxes
    pwin_indx = np.where((tstack>=np.min(twin_xz))&(tstack<=np.max(twin_xz)))[0]
    plt.plot([tstack[pwin_indx[0]],tstack[pwin_indx[-1]+1],tstack[pwin_indx[-1]+1],
              tstack[pwin_indx[0]],tstack[pwin_indx[0]]],
            [nwin,nwin,0,0,nwin],'c--',lw=3)
    if i==0:
        print('XZ window')
        print(tstack[pwin_indx[0]],tstack[pwin_indx[-1]+1])
#     plt.text(tstack[pwin_indx[-1]]*1.1,0.3*nwin,"dv/v\nwindow",
#                 fontsize=14,ha="left",va="center",color="g")
    
    if freqmin is not None and freqmax is not None:
        plt.title('%s %s-%s: %d km: %s' % (figlabels[i+2],netstachan1,netstachan2,
                                                                   dist,dlabels[i]),
                    fontsize=14,loc="left",pad=9)
        plt.text(-0.95*lag,nwin*0.9,"%4.1f-%4.1f Hz"%(freqmin,freqmax),backgroundcolor="w",
                fontsize=13)
    else:
        plt.title('%s %s-%s: %d km: %s' % (figlabels[i+2],netstachan1,netstachan2,dist,dlabels[i]),
                 fontsize=14,loc="left",pad=9)
        plt.text(0.35*lag,nwin*0.95,"unfiltered",backgroundcolor="w",fontsize=14)
    plt.xlabel('lag time (s)',fontsize=13)
    plt.xticks(t)
    plt.yticks(np.arange(0,nwin,step=tick_inc),labels=tmarks[0:nwin:tick_inc])
    plt.xlim([-lag,lag])
    plt.colorbar(orientation="horizontal",label="normalized amplitudes")

plt.tight_layout()
if savefig:
    plt.savefig("xcorr_examples_twonetworks."+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()

Cascadia window
86.0 106.0
XZ window
29.0 49.0


### Plot CPU time

In [9]:
step=5
step=5
nstart=step

c_temp=cdata_raw.subset(endtime="2012_04_1_0_0_0",overwrite=False)
ntrace=c_temp.data.shape[0]
stepall=np.arange(nstart,ntrace+0.5*step,step)

savefig=True
with open('xcorr_stacks_singlepair_cputime_step%d_%s.pk'%(step,network),'rb') as xf:
    stack_cputime=pickle.load(xf)
figsize=(12,4)
plt.figure(figsize=figsize,facecolor='w')
for i in range(len(dlabels)):
    plt.subplot(1,2,i+1)
    for j,m in enumerate(methods):
        plt.plot(stepall,stack_cputime[dlabels[i]][j,:],color=colors[j],label=method_labels[j])
    plt.grid("both")
    if i==1:plt.legend(bbox_to_anchor=(1.01, 1.0),ncol=1,fontsize=12)
    plt.yscale('log')
    plt.xlim([stepall[0],stepall[-1]])
#     plt.ylim([0.05,10**6])
    if i==0:plt.ylabel('CPU time (s)',fontsize=13)
    plt.xlabel('size of NCF subset',fontsize=13)
    plt.title(figlabels[i]+" "+dlabels[i],fontsize=14,loc='left',pad=9)
plt.tight_layout()
if savefig:
    plt.savefig("xcorr_stacks_singlepair_cputime_"+network+'.'+figformat,dpi=figdpi)
    plt.close()
else:
    plt.show()